# Segundo entrenamiento

## importar librerias

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import random
# Helper libraries
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras import optimizers
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.multiclass import OneVsRestClassifier
import os
from sklearn.metrics import confusion_matrix, classification_report 
import pandas as pd 
import seaborn as sn 
from collections import Counter

from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras import optimizers


## METODOS

In [2]:


def red(ArchivoEntrenamiento,ArchivoPrueba,base,numero_clases,epocas,lotes,bandas,pathResources="../../resources/",nameInfo="nombrePorDefecto"):
    #adquisicion de datos
    datosPrueba=np.loadtxt(ArchivoPrueba)
    datosEntrenamiento=np.loadtxt(str(ArchivoEntrenamiento))
    

    print(datosPrueba.shape)
    print(datosEntrenamiento.shape)

    #se separan los datos por la columna
    columnasD=datosEntrenamiento.shape[1]-1
    columnaC=datosEntrenamiento.shape[1]-1
    x1=datosEntrenamiento[:,0:columnasD]
    y1=datosEntrenamiento[:,columnaC]
    x2=datosPrueba[:,0:columnasD]
    y2=datosPrueba[:,columnaC]
    y1=y1.astype(int)
    y2=y2.astype(int)
    print(len(x1),len(x2),len(y1),len(y2))
    
    #se define la red, cambia para cada base de datos

    tf.keras.backend.clear_session() 
    adam=tf.keras.optimizers.Adam(learning_rate=0.001)
    capa_entrada= Input(shape=(bandas,))
    capa1= Dense(50, activation='relu')(capa_entrada)
    capa2= Dense(40, activation='relu')(capa1)
    capa3= Dense(30, activation='relu')(capa2)
    capa4= Dense(20, activation='relu')(capa3)
    capa_salida= Dense(numero_clases, activation='softmax')(capa4)
    salida = Model(inputs=capa_entrada,outputs=capa_salida)
    
    salida.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    snn=salida.fit(x1,y1,batch_size=lotes,epochs=(epocas),shuffle=True)

    #plt.plot(snn.history['acc'],label='acc') #Grafica los valores de accuracy por epoca
    fig=plt.plot(snn.history['loss'],label='loss')# Grafica los valores del error cuadratico medio por epoca
    sn.set(font_scale=.8) #for label size
    plt.legend(loc='center right',fontsize='10')
    plt.title(str("Grafica_"+nameInfo),fontsize='16')
    plt.savefig(pathResources+"Grafica_"+nameInfo, bbox_inches='tight')
    plt.close("all")
    #plt.show()
    final=datetime.datetime.now()

    evaluation = salida.evaluate(x1,y1,batch_size=100,verbose=1)#x1,y1,batch_size=100,verbose=1
    snn_pred = salida.predict(x1, batch_size=100) 
    snn_predicted = np.argmax(snn_pred, axis=1)

    #Creamos la matriz de confusión
    snn_cm = confusion_matrix(y1, snn_predicted) 

    #Normalizar matriz de confusión
    snn_cmN= np.zeros((len(snn_cm),len(snn_cm)))
    for i in range(len(snn_cm)):
        total=0
        for k in range(len(snn_cm)):
            total=total+snn_cm[i][k]
            total=total.astype(float)
        for j in range(len(snn_cm)):
            snn_cmN[i][j]=(snn_cm[i][j]/total)

    snn_report = classification_report(y1, snn_predicted,digits=4)
    print(snn_report)
    
    snn_df_cm = pd.DataFrame(snn_cmN, range(len(Counter(y1))), range(len(Counter(y1)))) ##range son el numero de clases
    #sn.set(font_scale=1.4) #for label size 
    sn.heatmap(snn_df_cm, annot=True) # font size 
    plt.title(str("Matriz de confucion Entrenamiento "+nameInfo))
    plt.savefig(pathResources+"Matriz_Confusion_Entrenamiento_"+nameInfo, bbox_inches='tight')
    plt.close("all")
    #plt.show()
    evaluation = salida.evaluate(x2,y2,batch_size=100,verbose=1)#x1,y1,batch_size=100,verbose=1
    snn_pred = salida.predict(x2, batch_size=100) 
    snn_predicted = np.argmax(snn_pred, axis=1)

    #Creamos la matriz de confusión
    snn_cm = confusion_matrix(y2, snn_predicted) 

    #Normalizar matriz de confusión
    snn_cmN= np.zeros((len(snn_cm),len(snn_cm)))
    for i in range(len(snn_cm)):
        total=0
        for k in range(len(snn_cm)):
            total=total+snn_cm[i][k]
            total=total.astype(float)
        for j in range(len(snn_cm)):
            snn_cmN[i][j]=(snn_cm[i][j]/total)

    # Visualiamos la matriz de confusión 
    snn_df_cm = pd.DataFrame(snn_cmN, range(len(Counter(y2))), range(len(Counter(y2)))) ##range son el numero de clases
    #sn.set(font_scale=1.4) #for label size 
    sn.heatmap(snn_df_cm, annot=True) # font size 
    plt.title(str("Matriz de confucion Test "+nameInfo))
    plt.savefig(pathResources+"Matriz_Confusion_Test_"+nameInfo, bbox_inches='tight')
    plt.close("all")
    #plt.show()

    snn_report1 = classification_report(y2, snn_predicted,digits=4)
    print(snn_report1)

    mult=1
    for i in range(numero_clases):
        mult=mult*snn_cmN[i][i]
    error=np.power(mult,1/numero_clases)
    print(error)
    #se almacena la informacion 
    info=open(str(pathResources+nameInfo+'.txt'),'w')
    info.write(os.linesep)
    info.write(os.linesep)
    info.write(os.linesep)
    info.write(snn_report)
    info.write(os.linesep)
    info.write(os.linesep)
    info.write(snn_report1)
    info.write(os.linesep)
    info.write(os.linesep)
    info.write(str(error))
    info.close()
    #se guarda el modero
    salida.save(str('../../modelos2/'+nameInfo+'.h5'))
    salida.save_weights("../../pesos2/"+nameInfo+'.weights.h5')
    del salida
    return error
  



## Iniciando proceso de segundo entrenamiento

### Parametros

In [3]:

archivosDeEntrenamiento = [
    "_TL", "_ENN", "_OSS", "_TL_SMOTE", "_ENN_SMOTE", "_OSS_SMOTE",  "_TL_ROS", "_ENN_ROS", "_OSS_ROS",
    "C1_TL", "C1_ENN", "C1_OSS", "C1_TL_SMOTE", "C1_ENN_SMOTE", "C1_OSS_SMOTE", "C1_TL_ROS", "C1_ENN_ROS", "C1_OSS_ROS",
    "C2_TL", "C2_ENN", "C2_OSS", "C2_TL_SMOTE", "C2_ENN_SMOTE", "C2_OSS_SMOTE", "C2_TL_ROS", "C2_ENN_ROS", "C2_OSS_ROS",
    "C3_TL", "C3_ENN", "C3_OSS", "C3_TL_SMOTE", "C3_ENN_SMOTE", "C3_OSS_SMOTE", "C3_TL_ROS", "C3_ENN_ROS", "C3_OSS_ROS",
    "C4_TL", "C4_ENN", "C4_OSS", "C4_TL_SMOTE", "C4_ENN_SMOTE", "C4_OSS_SMOTE", "C4_TL_ROS", "C4_ENN_ROS", "C4_OSS_ROS"
]



directorio="../../"
directorioPrueba=directorio+"Datasets/txt/test/"
directorioEntrenamiento=directorio+"Datasets/txt/sampled_train2/"
pathResources=None
bases=["Salinas"]
epocas=[500]
lotes=[500]
clases=[17]
bandas=[224]

In [4]:
counterDatasets=0
for base in bases:                                            
	ArchivoTest=directorioPrueba+base+".txt"
	for archivoDeEntrenamiento in archivosDeEntrenamiento:
		try:
			inicio=datetime.datetime.now()
			print(">>>>>>>  Iniciando proceso para el segundo entrenamiento con el archivo :",archivoDeEntrenamiento," base: ",base," con # de epocas: ",epocas[counterDatasets]," # de bandas: ",bandas[counterDatasets]," # lotes: ",lotes[counterDatasets]," y # clases: ",clases[counterDatasets])
			pathResources="../../resources/"
			nameInfo=base+"_"+archivoDeEntrenamiento
			archivoDeEntrenamiento=directorioEntrenamiento+base+archivoDeEntrenamiento+".txt"
			if(os.path.exists(archivoDeEntrenamiento)):

				if(os.path.exists(path='../../modelos2/'+nameInfo+'.h5')==False and os.path.exists(path="../../pesos2/"+nameInfo+'.weights.h5')==False):
					print(">> Archivo de entrenamiento encontrado: ",archivoDeEntrenamiento)
					acc=red(ArchivoEntrenamiento=archivoDeEntrenamiento,ArchivoPrueba=ArchivoTest,base=base,numero_clases=clases[counterDatasets],epocas=epocas[counterDatasets],lotes=lotes[counterDatasets],bandas=bandas[counterDatasets],pathResources=pathResources,nameInfo=nameInfo)
					fin=datetime.datetime.now()
					print(">> Entrenamiento: ",archivoDeEntrenamiento," terminado en: ",str(fin-inicio))
				else:
					print("<><><><> Modelo: "+nameInfo+"  ya entrenado<><><><>")

			else:
				print(">> Error: Archivo de entrenamiento: ",archivoDeEntrenamiento," no encontrado")
		except Exception as e:
			print("> Error en segundo entrenamiento con el archivo :",archivoDeEntrenamiento," base: ",base," con # de epocas: ",epocas[counterDatasets]," # de bandas: ",bandas[counterDatasets]," # lotes: ",lotes[counterDatasets]," y # clases: ",clases[counterDatasets])
			print(">>> Error: ",e)
	counterDatasets=counterDatasets+1

>>>>>>>  Iniciando proceso para el segundo entrenamiento con el archivo : _TL  base:  Salinas  con # de epocas:  500  # de bandas:  224  # lotes:  500  y # clases:  17
<><><><> Modelo: Salinas__TL  ya entrenado<><><><>
>>>>>>>  Iniciando proceso para el segundo entrenamiento con el archivo : _ENN  base:  Salinas  con # de epocas:  500  # de bandas:  224  # lotes:  500  y # clases:  17
<><><><> Modelo: Salinas__ENN  ya entrenado<><><><>
>>>>>>>  Iniciando proceso para el segundo entrenamiento con el archivo : _OSS  base:  Salinas  con # de epocas:  500  # de bandas:  224  # lotes:  500  y # clases:  17
<><><><> Modelo: Salinas__OSS  ya entrenado<><><><>
>>>>>>>  Iniciando proceso para el segundo entrenamiento con el archivo : _TL_SMOTE  base:  Salinas  con # de epocas:  500  # de bandas:  224  # lotes:  500  y # clases:  17
<><><><> Modelo: Salinas__TL_SMOTE  ya entrenado<><><><>
>>>>>>>  Iniciando proceso para el segundo entrenamiento con el archivo : _ENN_SMOTE  base:  Salinas  con # 

c:\Users\Daniel\anaconda3\envs\violence\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Daniel\anaconda3\envs\violence\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Daniel\anaconda3\envs\violence\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         1
           1     1.0000    1.0000    1.0000     37040
           2     1.0000    1.0000    1.0000     39056
           3     1.0000    0.9997    0.9999     39682
           4     0.9991    0.9999    0.9995     38275
           5     0.9997    0.9992    0.9995     39265
           6     1.0000    1.0000    1.0000     39862
           7     1.0000    1.0000    1.0000     37932
           8     0.9339    0.8976    0.9154     39563
           9     0.9964    0.9999    0.9981     37067
          10     0.9997    0.9950    0.9973     37576
          11     0.9999    1.0000    1.0000     37873
          12     1.0000    1.0000    1.0000     39106
          13     1.0000    0.9999    0.9999     38137
          14     0.9994    1.0000    0.9997     39701
          15     0.8997    0.9354    0.9172     38791
          16     0.9993    1.0000    0.9996     38170

    accuracy              